In [1]:
def wrapped(indexes_all):    
    import pickle
    import os
    import functools
    os.environ["XLA_PYTHON_CLIENT_MEM_FRACTION"]="0.8"
    import sys
    sys.path.insert(0, '../olympus')
    sys.path.insert(0, '../hyperion')
    import matplotlib as mpl
    import matplotlib.pyplot as plt
    import numpy as np
    import seaborn as sns
    from itertools import product
    import awkward as ak
    import pandas as pd
    from olympus.event_generation.photon_propagation.norm_flow_photons import make_generate_norm_flow_photons, make_nflow_photon_likelihood
    from olympus.event_generation.photon_propagation.utils import sources_to_model_input
    from olympus.event_generation.detector import make_hex_grid, Detector, make_line, make_triang, make_rhombus
    from olympus.event_generation.event_generation import (
        generate_cascade,
        generate_cascades,
        simulate_noise,
        generate_realistic_track,
        generate_realistic_tracks,
        generate_realistic_tracks_test,
        generate_realistic_starting_tracks,)
    from olympus.event_generation.lightyield import make_pointlike_cascade_source, make_realistic_cascade_source
    from olympus.event_generation.utils import sph_to_cart_jnp, proposal_setup
    
    #from olympus.plotting import plot_event
    from hyperion.medium import medium_collections
    from hyperion.constants import Constants
    import jax
    jax.config.update('jax_platform_name', 'cpu')
    from jax import random
    from jax import numpy as jnp
    import json


    event_type = indexes_all[0]
    v_x = indexes_all[1]
    seed_start = indexes_all[2]

    path_to_config = "../hyperion/data/pone_config_optimistic.json"
    config = json.load(open(path_to_config))["photon_propagation"]
    ref_ix_f, sca_a_f, sca_l_f, _ = medium_collections[config["medium"]]

    def c_medium_f(wl):
        """Speed of light in medium for wl (nm)."""
        return Constants.BaseConstants.c_vac / ref_ix_f(wl)

    rng = np.random.RandomState(31338)
    oms_per_line = 20
    dist_z = 50 # m
    dark_noise_rate = 16 * 1e-5  # 1/ns
    side_len = 100 # m
    pmts_per_module = 16
    pmt_cath_area_r = 75E-3 / 2 # m
    module_radius = 0.21 # m

    efficiency = pmts_per_module * (pmt_cath_area_r)**2 * np.pi / (4*np.pi*module_radius**2)
    det = make_triang(side_len, oms_per_line, dist_z, dark_noise_rate, rng, efficiency=efficiency, v_x=v_x, buoy_weight=30)

    gen_ph = make_generate_norm_flow_photons(
        "../hyperion/data/photon_arrival_time_nflow_params.pickle",
        "../hyperion/data/photon_arrival_time_counts_params.pickle",
        c_medium=c_medium_f(700) / 1E9

    )
    
    #
    prop = proposal_setup()
    events = generate_realistic_starting_tracks(
        det,
        cylinder_height=det._outer_cylinder[1] + 100,
        cylinder_radius=det._outer_cylinder[0] + 50,
        log_emin=3.5,
        log_emax=6.5,
        nsamples=100,
        seed=seed_start,
        pprop_func=gen_ph,
        proposal_prop=prop

    )
        
    return events






In [1]:
import sys
sys.path.insert(0, '../olympus')
sys.path.insert(0, '../hyperion')
from multiprocessing import Pool
import jax
jax.config.update('jax_platform_name', 'cpu')
import pickle
from nemesis.event_generation.generate_multiCPU import events_wrapped

In [ ]:
first_seed = 2000000
for it in range(1,10):
    indexes_all = []

    seed_start = first_seed + 1600*it
    event_type = 'cascades'
    v_x = (1 + it)/100
    #v_x = 0
    n_proc = 15
    n_events_per_process = 100
    seed_starts = range(seed_start, seed_start+n_proc*n_events_per_process, n_events_per_process)
    for v in seed_starts:
        indexes_all.append([event_type, v_x, v])
    
    if __name__ == '__main__':
        with Pool(n_proc) as p:
            all_res = p.map(events_wrapped, indexes_all)
    
    events = []
    records = []
    for i in range(0, len(all_res)):
        for j in range(100):
            events.append(all_res[i][0][j])
            records.append(all_res[i][1][j])
    pickle.dump((events, records), open(f"/dss/pone/pone_events/cascades{len(events)}_vx{v_x}_{it+30}.pickle", "wb"))
    seed_start += 1600
    print(f'iteration number {it} completed!')

/home/work/.local/lib/python3.10/site-packages/jax/_src/api_util.py:222: SyntaxWarning: Jitted function has invalid argnames {'interactions'} in static_argnames. Function does not take these args.This warning will be replaced by an error after 2022-08-20 at the earliest.
  warnings.warn(f"Jitted function has invalid argnames {invalid_argnames} "


  0%|          | 0/100 [00:00<?, ?it/s]

/home/work/.local/lib/python3.10/site-packages/jax/_src/api_util.py:222: SyntaxWarning: Jitted function has invalid argnames {'interactions'} in static_argnames. Function does not take these args.This warning will be replaced by an error after 2022-08-20 at the earliest.
  warnings.warn(f"Jitted function has invalid argnames {invalid_argnames} "


  0%|          | 0/100 [00:00<?, ?it/s]

/home/work/.local/lib/python3.10/site-packages/jax/_src/api_util.py:222: SyntaxWarning: Jitted function has invalid argnames {'interactions'} in static_argnames. Function does not take these args.This warning will be replaced by an error after 2022-08-20 at the earliest.
  warnings.warn(f"Jitted function has invalid argnames {invalid_argnames} "


...

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

In [3]:
first_seed = 2000000

for it in range(10):  
    seed_start = first_seed + 1600*it
    n_proc = 15
    n_events_per_process = 100
    seed_starts = range(seed_start, seed_start+n_proc*n_events_per_process, n_events_per_process)

    if __name__ == '__main__':
        with Pool(n_proc) as p:
            all_res = p.map(wrapped, seed_starts)

    events = []
    records = []
    for i in range(0, len(all_res)):
        for j in range(100):
            events.append(all_res[i][0][j])
            records.append(all_res[i][1][j])
    pickle.dump((events, records), open(f"/dss/pone/pone_events/stracks_{len(events)}_{it+30}.pickle", "wb"))
    seed_start += 1600
    print(f'iteration number {it} completed!')

TypeError: 'int' object is not subscriptable

In [2]:
7+3

10

In [1]:
def wrapped(seed_start):    
    import pickle
    import os
    import functools
    os.environ["XLA_PYTHON_CLIENT_MEM_FRACTION"]="0.8"
    import sys
    sys.path.insert(0, '../olympus')
    sys.path.insert(0, '../hyperion')
    import matplotlib as mpl
    import matplotlib.pyplot as plt
    import numpy as np
    import seaborn as sns
    from itertools import product
    import awkward as ak
    import pandas as pd
    from olympus.event_generation.photon_propagation.norm_flow_photons import make_generate_norm_flow_photons, make_nflow_photon_likelihood
    from olympus.event_generation.photon_propagation.utils import sources_to_model_input
    from olympus.event_generation.detector import make_hex_grid, Detector, make_line, make_triang, make_rhombus
    from olympus.event_generation.event_generation import (
        generate_cascade,
        generate_cascades,
        simulate_noise,
        generate_realistic_track,
        generate_realistic_tracks,
        generate_realistic_tracks_test,
        generate_realistic_starting_tracks,)
    from olympus.event_generation.lightyield import make_pointlike_cascade_source, make_realistic_cascade_source
    from olympus.event_generation.utils import sph_to_cart_jnp, proposal_setup
    #from olympus.plotting import plot_event
    from hyperion.medium import medium_collections
    from hyperion.constants import Constants
    import jax
    jax.config.update('jax_platform_name', 'cpu')
    from jax import random
    from jax import numpy as jnp
    import json



#wrapped = lambda seed_start: generate_cascades(
#    det,
#    cylinder_height=det._outer_cylinder[1] + 1100,
#    cylinder_radius=det._outer_cylinder[0] + 50,
#    log_emin=3.5,
#    log_emax=6.5,
#    particle_id=11,
#    nsamples=n_events_per_process,
#    seed=seed_start,
#    converter_func=functools.partial(make_realistic_cascade_source, moliere_rand=True, resolution=0.2),
#    pprop_func=gen_ph)
#
#print(wrapped)



    path_to_config = "../hyperion/data/pone_config_optimistic.json"
    config = json.load(open(path_to_config))["photon_propagation"]
    ref_ix_f, sca_a_f, sca_l_f, _ = medium_collections[config["medium"]]

    def c_medium_f(wl):
        """Speed of light in medium for wl (nm)."""
        return Constants.BaseConstants.c_vac / ref_ix_f(wl)


    rng = np.random.RandomState(31338)
    oms_per_line = 20
    dist_z = 50 # m
    dark_noise_rate = 16 * 1e-5  # 1/ns
    side_len = 100 # m
    pmts_per_module = 16
    pmt_cath_area_r = 75E-3 / 2 # m
    module_radius = 0.21 # m

    # Calculate the relative area covered by PMTs
    efficiency = pmts_per_module * (pmt_cath_area_r)**2 * np.pi / (4*np.pi*module_radius**2)
    det = make_triang(side_len, oms_per_line, dist_z, dark_noise_rate, rng, efficiency=efficiency)

    gen_ph = make_generate_norm_flow_photons(
        "../hyperion/data/photon_arrival_time_nflow_params.pickle",
        "../hyperion/data/photon_arrival_time_counts_params.pickle",
        c_medium=c_medium_f(700) / 1E9

    )
    prop = proposal_setup()
    events = generate_realistic_tracks(
    det,
    cylinder_height=det._outer_cylinder[1] + 100,
    cylinder_radius=det._outer_cylinder[0] + 50,
    log_emin=3.5,
    log_emax=6.5,
    nsamples=100,
    seed=seed_start,
    pprop_func=gen_ph,
    proposal_prop=prop
    )
        
    return events



import sys
sys.path.insert(0, '../olympus')
sys.path.insert(0, '../hyperion')
from multiprocessing import Pool
import jax
jax.config.update('jax_platform_name', 'cpu')
import pickle


In [ ]:
seed_start_fisrt = 1000009
for it in range(7,10):
    seed_start = seed_start_fisrt + 1600*it
    n_proc = 15
    n_events_per_process = 100
    seed_starts = range(seed_start, seed_start+n_proc*n_events_per_process, n_events_per_process)

    if __name__ == '__main__':
        with Pool(n_proc) as p:
            all_res = p.map(wrapped, seed_starts)

    events = []
    records = []
    for i in range(0, len(all_res)):
        for j in range(100):
            events.append(all_res[i][0][j])
            records.append(all_res[i][1][j])
    pickle.dump((events, records), open(f"/dss/pone/pone_events/tracks_{len(events)}_{it+30}.pickle", "wb"))
    print(f'iteration number {it} completed!')

/home/work/.local/lib/python3.10/site-packages/jax/_src/api_util.py:222: SyntaxWarning: Jitted function has invalid argnames {'interactions'} in static_argnames. Function does not take these args.This warning will be replaced by an error after 2022-08-20 at the earliest.
  warnings.warn(f"Jitted function has invalid argnames {invalid_argnames} "


  0%|          | 0/100 [00:00<?, ?it/s]

/home/work/.local/lib/python3.10/site-packages/jax/_src/api_util.py:222: SyntaxWarning: Jitted function has invalid argnames {'interactions'} in static_argnames. Function does not take these args.This warning will be replaced by an error after 2022-08-20 at the earliest.
  warnings.warn(f"Jitted function has invalid argnames {invalid_argnames} "
/home/work/.local/lib/python3.10/site-packages/jax/_src/api_util.py:222: SyntaxWarning: Jitted function has invalid argnames {'interactions'} in static_argnames. Function does not take these args.This warning will be replaced by an error after 2022-08-20 at the earliest.
  warnings.warn(f"Jitted function has invalid argnames {invalid_argnames} "


  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

...

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

In [1]:
def wrapped(seed_start):    
    import pickle
    import os
    import functools
    os.environ["XLA_PYTHON_CLIENT_MEM_FRACTION"]="0.8"
    import sys
    sys.path.insert(0, '../../olympus')
    sys.path.insert(0, '../../hyperion')
    import matplotlib as mpl
    import matplotlib.pyplot as plt
    import numpy as np
    import seaborn as sns
    from itertools import product
    import awkward as ak
    import pandas as pd
    from olympus.event_generation.photon_propagation.norm_flow_photons import make_generate_norm_flow_photons, make_nflow_photon_likelihood
    from olympus.event_generation.photon_propagation.utils import sources_to_model_input
    from olympus.event_generation.detector import make_hex_grid, Detector, make_line, make_triang, make_rhombus
    from olympus.event_generation.event_generation import (
        generate_cascade,
        generate_cascades,
        simulate_noise,
        generate_realistic_track,
        generate_realistic_tracks,
        generate_realistic_tracks_test,
        generate_realistic_starting_tracks,)
    from olympus.event_generation.lightyield import make_pointlike_cascade_source, make_realistic_cascade_source
    from olympus.event_generation.utils import sph_to_cart_jnp, proposal_setup
    #from olympus.plotting import plot_event
    from hyperion.medium import medium_collections
    from hyperion.constants import Constants
    import jax
    jax.config.update('jax_platform_name', 'cpu')
    from jax import random
    from jax import numpy as jnp
    import json



    path_to_config = "../../hyperion/data/pone_config_optimistic.json"
    config = json.load(open(path_to_config))["photon_propagation"]
    ref_ix_f, sca_a_f, sca_l_f, _ = medium_collections[config["medium"]]

    def c_medium_f(wl):
        """Speed of light in medium for wl (nm)."""
        return Constants.BaseConstants.c_vac / ref_ix_f(wl)


    rng = np.random.RandomState(31338)
    oms_per_line = 20
    dist_z = 50 # m
    dark_noise_rate = 16 * 1e-5  # 1/ns
    side_len = 100 # m
    pmts_per_module = 16
    pmt_cath_area_r = 75E-3 / 2 # m
    module_radius = 0.21 # m

    # Calculate the relative area covered by PMTs
    efficiency = pmts_per_module * (pmt_cath_area_r)**2 * np.pi / (4*np.pi*module_radius**2)
    det = make_triang(side_len, oms_per_line, dist_z, dark_noise_rate, rng, efficiency=efficiency)

    gen_ph = make_generate_norm_flow_photons(
        "../../hyperion/data/photon_arrival_time_nflow_params.pickle",
        "../../hyperion/data/photon_arrival_time_counts_params.pickle",
        c_medium=c_medium_f(700) / 1E9

    )
    #
    events = generate_cascades(
        det,
        cylinder_height=det._outer_cylinder[1] + 1100,
        cylinder_radius=det._outer_cylinder[0] + 50,
        log_emin=3.5,
        log_emax=6.5,
        particle_id=11,
        nsamples=100,
        seed=seed_start,
        converter_func=functools.partial(make_realistic_cascade_source, moliere_rand=True, resolution=0.2),
        pprop_func=gen_ph,
        noise_function=None

    )
        
    return events



import sys
sys.path.insert(0, '../../olympus')
sys.path.insert(0, '../../hyperion')
from multiprocessing import Pool
import jax
jax.config.update('jax_platform_name', 'cpu')
import pickle


In [2]:
#seed_start = 1100000
for it in range(8, 10):    
    seed_start = 1100005 + 1600*it
    n_proc = 15
    n_events_per_process = 100
    seed_starts = range(seed_start, seed_start+n_proc*n_events_per_process, n_events_per_process)

    if __name__ == '__main__':
        with Pool(n_proc) as p:
            all_res = p.map(wrapped, seed_starts)

    events = []
    records = []
    for i in range(0, len(all_res)):
        for j in range(100):
            events.append(all_res[i][0][j])
            records.append(all_res[i][1][j])
    pickle.dump((events, records), open(f"/dss/pone/pone_events/cascades_{len(events)}_{it+30}.pickle", "wb"))
    seed_start += 1600
    print(f'iteration number {it} completed!')

/home/work/.local/lib/python3.10/site-packages/jax/_src/api_util.py:222: SyntaxWarning: Jitted function has invalid argnames {'interactions'} in static_argnames. Function does not take these args.This warning will be replaced by an error after 2022-08-20 at the earliest.
  warnings.warn(f"Jitted function has invalid argnames {invalid_argnames} "
/home/work/.local/lib/python3.10/site-packages/jax/_src/api_util.py:222: SyntaxWarning: Jitted function has invalid argnames {'interactions'} in static_argnames. Function does not take these args.This warning will be replaced by an error after 2022-08-20 at the earliest.
  warnings.warn(f"Jitted function has invalid argnames {invalid_argnames} "
/home/work/.local/lib/python3.10/site-packages/jax/_src/api_util.py:222: SyntaxWarning: Jitted function has invalid argnames {'interactions'} in static_argnames. Function does not take these args.This warning will be replaced by an error after 2022-08-20 at the earliest.
  warnings.warn(f"Jitted functio

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

...

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

iteration number 9 completed!
